In [1]:
import os
import copy
import numpy as np
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics    
from pydrake.trajectories import PiecewisePolynomial

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import running_as_notebook

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
def load_obj(filename):
    vertices = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.split()
                vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
    return np.array(vertices)

def get_dimensions(vertices):
    min_vals = np.min(vertices, axis=0)
    max_vals = np.max(vertices, axis=0)
    return max_vals - min_vals

In [4]:
def read_obj(
        obj_name,
        obj_position,
        obj_rotation,
        obj_mass = 0.5,
        obj_dir = os.path.join(os.path.dirname(os.getcwd()), "OBJs")
):
  obj_path = os.path.join(obj_dir, "{}.obj".format(obj_name))
  x, y, z = obj_position
  roll, pitch, yaw = obj_rotation
  obj_link = "file://{}".format(obj_path)
  # Load the OBJ file
  vertices = load_obj(obj_path)
  # Calculate dimensions
  dimensions = get_dimensions(vertices)
  width, height, depth = dimensions
  object_sdf = """
  <?xml version="1.0"?>
  <sdf version="1.7">
    <model name="{}">
      <link name="base_link_sugar">
        <inertial>
          <mass>{}</mass>
          <inertia>
            <ixx>0.001418</ixx>
            <ixy>0</ixy>
            <ixz>0</ixz>
            <iyy>0.000455</iyy>
            <iyz>0</iyz>
            <izz>0.001699</izz>
          </inertia>
        </inertial>
        <visual name="base_link_sugar">
          <pose>{} {} {} {} {} {}</pose>
          <geometry>
            <mesh>
              <uri>{}</uri>
            </mesh>
          </geometry>
        </visual>
      <collision name="box_collision">
          <geometry>
            <box>
              <size>{} {} {}</size>
            </box>
          </geometry>
          <drake:proximity_properties>
            <drake:compliant_hydroelastic/>
            <drake:hydroelastic_modulus>1.0e8</drake:hydroelastic_modulus>
            ]
          </drake:proximity_properties>
        </collision>
      </link>
    </model>
  </sdf>
  """.format(obj_name, obj_mass, x, y, z, roll, pitch, yaw, obj_link, width, height, depth)

  sdf_dir = os.path.join(os.path.dirname(os.getcwd()), "obj_sdfs")
  object_sdf_file = os.path.join(sdf_dir, "{}.sdf".format(obj_name))
  with open(object_sdf_file, "w") as f:
      f.write(object_sdf)
  obj_description = """- add_model:
    name: {}
    file: "file://{}"
""".format(obj_name, object_sdf_file)
  return [width, height, depth], obj_description

In [5]:
table_name = "table"
table_bounds, table_description = read_obj(table_name, [0, 0, 0], [0, 0, 0])
box_name = "solid_box"
box_bounds, box_description = read_obj(box_name, [0, 0, table_bounds[2]], [0, 0, 0])
all_descriptions = [table_description, box_description]

In [6]:
# def read_sdf(obj_name, sdf_dir, obj_position, obj_rotation):
#   x, y, z = obj_position
#   roll, pitch, yaw = obj_rotation
#   obj_path = os.path.join(sdf_dir, "{}.sdf".format(obj_name))
#   sdf_link = "file://{}".format(obj_path)
#   obj_description = """- add_model:
#     name: {}
#     file: {}
# - add_weld:
#     parent_body: world
#     child: {}::base
#     X_PC:
#       translation: [{}, {}, {}].format(obj_name, sdf_link)
#       rotation: !Rpy {{deg : {}}}
# """.format(obj_name, sdf_link, obj_name, x, y, z, obj_rotation)
#   return obj_description

In [7]:
# num_objs = 3
# ycb_dir = os.path.join(os.getcwd(), "ycb")
# model_names = os.listdir(ycb_dir)
# selected_model_names = np.random.choice(model_names, num_objs, replace=False)
# for model_name in selected_model_names:
#     sdf_dir = os.path.join(ycb_dir, model_name)
#     files = os.listdir(sdf_dir)
#     for file in files:
#         if file.endswith(".sdf"):
#             obj_name = file.split(".")[0]
#             obj_description = read_sdf(obj_name, sdf_dir, [0, 0, table_bounds[2]], [0, 0, 0])
#             print(obj_description)
#             all_descriptions.append(obj_description)

In [8]:
yaml_file = """
directives:
"""
for description in all_descriptions:
    yaml_file += description
yaml_dir = os.path.join(os.path.dirname(os.getcwd()), "yaml")
if not os.path.exists(yaml_dir):
    os.makedirs(yaml_dir)
scene_path = os.path.join(yaml_dir, "scene.dmd.yaml")
with open(scene_path, "w") as f:
    f.write(yaml_file)

In [9]:
def setup_manipulation_station(yaml_path):
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="file://{}".format(yaml_path),
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")

    MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )


    diagram = builder.Build()

    context = plant.CreateDefaultContext()


    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)

In [10]:
def create_gripper_yaml(gripper_path):
    gripper_yaml = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [1]
        iiwa_joint_2: [1.6]
        iiwa_joint_3: [1]
        iiwa_joint_4: [-0.75]
        iiwa_joint_5: [1]
        iiwa_joint_6: [ 2.0]
        iiwa_joint_7: [1]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: file:///{}
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [00, 0, 0.114]
        rotation: !Rpy {{deg : [90, 0, 90]}}
""".format(gripper_path)
    return gripper_yaml
gripper_sdf_path = os.path.join(os.path.dirname(os.getcwd()), "gripper_sdfs", "schunk_wsg_50_hydro_bubble.sdf")
gripper_yaml = create_gripper_yaml(gripper_sdf_path)
gripper_yaml_path = os.path.join(os.path.dirname(os.getcwd()), "yaml", "gripper.dmd.yaml")
with open(gripper_yaml_path, "w") as f:
    f.write(gripper_yaml)

In [11]:
def setup_robot(path):
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename="file://{}".format(path),
            time_step=1e-3,
        )
    )
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(
        wsg_position.get_output_port(), station.GetInputPort("wsg_position")
    )

    diagram = builder.Build()

    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)

    return initial_pose

In [12]:
setup_manipulation_station(scene_path)
initial_pose = setup_robot(gripper_yaml_path)